## Module development

### Annotator

In [ ]:
from oakvar import BaseAnnotator
class Annotator(BaseAnnotator):
    def annotate(self, input_data: dict):
        out_data = {}
        if input_data["pos"] % 2 == 0:
            out_data["parity"] = "even"
        else:
            out_data["parity"] = "odd"
        return out_data

In [ ]:
awesome_module = Annotator(
    name="awesome", 
    title="Awesome annotator", 
    level="variant", 
    code_version="0.1.0", 
    input_file="exampleinput.crv", 
    output_columns=[
    {
        "name": "parity",
        "type": "string",
        "title": "Parity"
    }])

In [ ]:
awesome_module.annotate({"pos": 11})

In [ ]:
awesome_module.save(overwrite=True, interactive=True)

In [ ]:
m.run()

In [ ]:
!ov module ls awesome

In [ ]:
!ov module info awesome

In [ ]:
!mkdir data

In [ ]:
import sqlite3
conn = sqlite3.connect("data/awesome.sqlite")
cursor = conn.cursor()
cursor.execute("drop table if exists scores")
cursor.execute("create table scores "
    "(chrom text, pos int, alt_base text, score float)")
cursor.execute("insert into scores "
    "(chrom, pos, alt_base, score) "
    "values ('chr10', 87864470, 'T', 0.897)")
cursor.execute("insert into scores "
    "(chrom, pos, alt_base, score) 
    "values ('chr1', 12666560, 'A', 0.435)")
cursor.execute("create index scores_idx on scores (chrom, pos)")
conn.commit()

In [ ]:
def annotate(self, input_data: dict):
    out_data = {}
    self.cursor.execute(
        "select score from scores where chrom=? and pos=? and alt_base=?", 
        (input_data["chrom"], input_data["pos"], input_data["alt_base"])
    )
    hit = self.cursor.fetchone()
    if not hit:
        return None
    else:
        score = hit[0]
        return {"score": score}

In [ ]:
def query_db(self, chrom: str, pos: int, alt_base: str):
    self.cursor.execute(
        "select score from scores where chrom=? and pos=? and alt_base=?", 
        (chrom, pos, alt_base)
    )
    hit = self.cursor.fetchone()
    return hit

def annotate(self, input_data: dict):
    out_data = {}
    hit = self.query_db(
        input_data["chrom"], input_data["pos"], input_data["alt_base"])
    if not hit:
        return None
    else:
        score = hit[0]
        return {"score": score}

awesome_module.add_method(query_db)
awesome_module.add_method(annotate)

In [ ]:
awesome_module.set_output_columns([
    {"name": "score", "type": "float", "title": "Score"}
])

In [ ]:
awesome_module.base_setup()

In [ ]:
awesome_module.annotate({"chrom": "chr10", "pos": 87864470, "alt_base": "T"})

In [ ]:
awesome_module.run()

In [ ]:
awesome_module.save(overwrite=True)

In [ ]:
df=ov.read_crv("exampleinput.crv")

In [ ]:
def annotate_df(self, df):
    import polars as pl
    return df.with_columns(pl.col("pos").apply(lambda x: "even" if x % 2 == 0 else "odd").alias("col1"))    

In [ ]:
m.add_method(annotate_df)

In [ ]:
m.run_df(df)

### Converter

In [ ]:
from oakvar import BaseConverter
class Converter(BaseConverter):
    def convert_line(self, line):
        words = line.split()
        [chrom, pos, strand, ref_base, alt_base] = words[:5]
        out_data = {"chrom":chrom, "pos": pos, "ref_base": ref_base, "alt_base": alt_base}
        return out_data

In [ ]:
c = Converter(name="awesome-converter", code_version="0.1.0", title="Awesome Converter")

In [ ]:
f = open("exampleinput")

In [ ]:
for d in c.convert_file(f):
    print(d)

In [ ]:
c.save(overwrite=True)

In [ ]:
from oakvar import MasterConverter
mc = MasterConverter(["exampleinput"])
mc.run()

In [ ]:
inspect.linecache.getlines(inspect.getfile(Annotator))

In [ ]:
print("".join(inspect.getsourcelines(Annotator)[0]))

In [ ]:
from IPython.core.magics.code import extract_symbols

In [ ]:
cell_code = "".join(inspect.linecache.getlines(new_getfile(Annotator)))

In [ ]:
cell_code

In [ ]:
m = Annotator(module_name="awesome", level="variant", output_columns=[{"name": "col1", "type": "string", "title": "Col1"}])

In [ ]:
with open("text.py", "w") as wf:
    wf.write(str(Annotator))

In [ ]:
print(type(Annotator))

In [ ]:
inspect.getsourcelines(ov.BaseAnnotator)

In [ ]:
__main__.__file__ = "test.py"

In [ ]:
class A:
    pass
print(type(A))

In [ ]:
import inspect
inspect.getsource(m)

In [ ]:
c=ov.get_live_annotator(module_name="clinvar")

In [ ]:
c.annotate({"chrom": "chr11", "pos": 64810053, "ref_base": "-", "alt_base": "A"})